# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


In [102]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
#print(g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [12]:
cities_df = pd.read_csv("./output_data/cities.csv")
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,Caála,-12.8525,15.5606,55.69,35,5,1.12,AO,1627864105
1,Port Hawkesbury,45.6169,-61.3485,65.32,80,1,1.01,CA,1627864105
2,Praia da Vitória,38.7333,-27.0667,74.75,100,90,14.50,PT,1627864106
3,Dikson,73.5069,80.5464,40.01,99,99,7.78,RU,1627864106
4,Hilo,19.7297,-155.0900,83.93,67,40,12.66,US,1627864106


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [91]:
# Store latitude and longitude in locations
locations = cities_df[['Lat', 'Lng']]
locations.dtypes

weights = cities_df["Humidity"]

# # Plot Heatmap
fig = gmaps.figure(center=(50.0, -35.0), zoom_level=2)

# # Create heat layer
heatmap_layer = gmaps.heatmap_layer(locations, weights = weights,
                                   dissipating = False, max_intensity = 150, point_radius = 3)

# # Add layer
fig.add_layer(heatmap_layer)

# # Display figure
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [100]:
hotel_df = cities_df.loc[(cities_df['Max Temp'].between(70, 80))&(cities_df["Windspeed"] <10)& \
(cities_df["Cloudiness"] ==0)]
hotel_df



,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
15,Saskylakh,71.9167,114.0833,76.60,40,0,4.34,RU,1627864110
160,Abu Samrah,35.3029,37.1841,77.86,45,0,6.64,SY,1627863934
245,Nova Olímpia,-14.7972,-57.2881,72.50,32,0,3.15,BR,1627864080
247,Pontes e Lacerda,-15.2261,-59.3353,70.84,31,0,5.68,BR,1627864198
339,Damietta,31.4165,31.8133,79.65,67,0,6.08,EG,1627864237
379,Gaoua,10.2992,-3.2508,71.67,88,0,4.43,BF,1627864253
468,Pula,44.8683,13.8481,74.26,82,0,3.44,HR,1627864002
506,Mahon,39.8885,4.2658,70.36,78,0,4.61,ES,1627864304
513,Neyshabur,36.2133,58.7958,71.65,16,0,2.53,IR,1627864307
516,Boca do Acre,-8.7522,-67.3978,73.67,50,0,3.56,BR,1627864308


In [55]:
vac_temp = cities_df.loc[cities_df.loc[:, 'Max Temp'].between(70, 80)]
vac_wind = vac_temp.loc[vac_temp["Windspeed"] <10]
hotel_df = vac_wind.loc[vac_wind["Cloudiness"] ==0]
hotel_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
15,Saskylakh,71.9167,114.0833,76.60,40,0,4.34,RU,1627864110
160,Abu Samrah,35.3029,37.1841,77.86,45,0,6.64,SY,1627863934
245,Nova Olímpia,-14.7972,-57.2881,72.50,32,0,3.15,BR,1627864080
247,Pontes e Lacerda,-15.2261,-59.3353,70.84,31,0,5.68,BR,1627864198
339,Damietta,31.4165,31.8133,79.65,67,0,6.08,EG,1627864237
379,Gaoua,10.2992,-3.2508,71.67,88,0,4.43,BF,1627864253
468,Pula,44.8683,13.8481,74.26,82,0,3.44,HR,1627864002
506,Mahon,39.8885,4.2658,70.36,78,0,4.61,ES,1627864304
513,Neyshabur,36.2133,58.7958,71.65,16,0,2.53,IR,1627864307
516,Boca do Acre,-8.7522,-67.3978,73.67,50,0,3.56,BR,1627864308


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [82]:
import pprint

In [85]:

hotel_df['Hotel Name'] = ""
hotel_df

target_type = "Hotel"
radius = 5000

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# rewrite params dict
params = {
    "types": target_type,
    "radius": radius,
    "key": g_key
}

for index, row in hotel_df.iterrows():
    print(index)
    print(row)

#     # get City from df
    city_check = row['City']
    lat = row["Lat"]
    lng = row["Lng"]

#     # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

#     # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_check}.")
    print(requests.get(base_url, params=params).json())
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    #print(results)
    pprint.pprint(results)
   
    try:
        print(f"In {city_check} the closest hotel is {results[0]['name']}.")
        
        #DF empty
        hotel_df.loc[index,'Hotel Name'] = results[0]['name'] #API
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


#hotel_df

<ipython-input-85-91bd2b46dae6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


15
City           Saskylakh
Lat              71.9167
Lng             114.0833
Max Temp            76.6
Humidity              40
Cloudiness             0
Windspeed           4.34
Country               RU
Date          1627864110
Hotel Name              
Name: 15, dtype: object
Retrieving Results for Index 15: Saskylakh.
{'html_attributions': [], 'results': [{'geometry': {'location': {'lat': 71.954308, 'lng': 114.119835}, 'viewport': {'northeast': {'lat': 71.95776415008287, 'lng': 114.1358424237714}, 'southwest': {'lat': 71.95085120990271, 'lng': 114.1038275762286}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png', 'icon_background_color': '#7B9EB0', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_pinlet', 'name': 'Saskylakh', 'photos': [{'height': 2522, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/117971875721719968449">Юрий Исаев</a>'], 'photo_reference': 'Aap_uEC8b6bfQRepwZu6604Qgu

/Users/jenniferrocha/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


{'html_attributions': [], 'results': [{'geometry': {'location': {'lat': 35.3203237, 'lng': 37.2048894}, 'viewport': {'northeast': {'lat': 35.32510488263767, 'lng': 37.21215248241005}, 'southwest': {'lat': 35.31379449678393, 'lng': 37.19593045627117}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png', 'icon_background_color': '#7B9EB0', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_pinlet', 'name': 'Suruj', 'photos': [{'height': 960, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/108672083767378111716">أبـ,ـنـ,ـ سـ,ـۅريـ,ـآ</a>'], 'photo_reference': 'Aap_uEBx0HeRWRW05MCFFwFYU2w_StU57QxCICgj80fQ_EdznoePdQR4O7bCDKqpK62qbmn3Y8W_GgU_JBDJ8cTjAl_ZeEomw0S89x3CzGi4fh3k9hSWLfAKnvcvNLa2QAe0kCWQX4EWpVTW9hnpdaAEySHqOEvzvwdZ_u-8c0ENe9uYhTzT', 'width': 1280}], 'place_id': 'ChIJBQl_gmbJJBURG_l4aOr_f9M', 'reference': 'ChIJBQl_gmbJJBURG_l4aOr_f9M', 'scope': 'GOOGLE', 'types': ['locality', 'political']

               'viewport': {'northeast': {'lat': 35.30352703029151,
                                          'lng': 37.18763428029149},
                            'southwest': {'lat': 35.30082906970851,
                                          'lng': 37.1849363197085}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/school-71.png',
  'icon_background_color': '#7B9EB0',
  'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/school_pinlet',
  'name': 'مدرسة الوسيطة الإبتدائية',
  'place_id': 'ChIJCU36t_rJJBURYPhfkmt4ahs',
  'plus_code': {'compound_code': '852P+QG الوسيطة، Syria',
                'global_code': '8G7V852P+QG'},
  'reference': 'ChIJCU36t_rJJBURYPhfkmt4ahs',
  'scope': 'GOOGLE',
  'types': ['school', 'point_of_interest', 'establishment'],
  'vicinity': 'Unnamed Road, الوسيطة،'},
 {'business_status': 'OPERATIONAL',
  'geometry': {'location': {'lat': 35.3178856, 'lng': 37.20250860000001},
               'viewport': {'northeast

...

               'viewport': {'northeast': {'lat': 39.8860869802915,
                                          'lng': 4.280049280291502},
                            'southwest': {'lat': 39.8833890197085,
                                          'lng': 4.277351319708497}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
  'icon_background_color': '#909CE1',
  'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
  'name': 'Hotel Del Almirante',
  'photos': [{'height': 1080,
              'html_attributions': ['<a '
                                    'href="https://maps.google.com/maps/contrib/109073688623304845373">Christian '
                                    'Verdaguer Casals</a>'],
              'photo_reference': 'Aap_uECSOqyMYZQzK2xpci5izAnf5PcctMsorJcsHQT-mRjOzgg3W94hy3MIxSDQcFoiy9b-eER-ibKp0Dz-mptt4NnPB3RpS9f9SLUJA9ku00Qpz1X0wZb9oEcZ0kNvXWLAtS-t8HxaxLaF9MN6rrj_4Ub5l2mEQCUYMFtNO91k0lBXWtIt',
         

{'html_attributions': [], 'next_page_token': 'Aap_uEBioFcG9k5ex6r3Bu4EnjCXrrOtRZRi60LDQxoS5dA38gGzXi42BELw-bNTtu4FDP7HYgTz_ock2E-dhK9T0R4NKIEOXSzkFbs7XePMth39QfuO5ELInrXHIfywSoWTWtTi7MbcWaLZvWbUbDxiBWAVXMrvNSGLln6d0I6W4Cz8X3TF08nHiNnxrEGzyG-0xUKwxWzs8sXi8CDoEKDs_aG8GuQ7feLUOM0FmVT8mahFbUDLiDfJSU7L-3xu7HUgnaiTslHEff2JeLyk3601ALvqNyWyQsLSRWwioV8mu-hoa_MA0FkOJxNx_nuMU_duFdepgIcKj6B8jIvMpu6dZbY51OPk6pAQpgMrP5FhBTjCFi-uaZ0DPvbOEspAqMgwGqjgAGN7xuCzUyzyeBQ9WIRBKo3yEbKhWmcqSgbfy9LSM-BMyWNeag', 'results': [{'geometry': {'location': {'lat': 36.2140865, 'lng': 58.7960915}, 'viewport': {'northeast': {'lat': 36.29881990412616, 'lng': 58.85179897319068}, 'southwest': {'lat': 36.16420955310242, 'lng': 58.73158893629884}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png', 'icon_background_color': '#7B9EB0', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_pinlet', 'name': 'Neyshabur', 'photos': [{'height': 4128, 'html_attri

 {'business_status': 'OPERATIONAL',
  'geometry': {'location': {'lat': 36.2073, 'lng': 58.79084649999999},
               'viewport': {'northeast': {'lat': 36.2084887802915,
                                          'lng': 58.79227923029151},
                            'southwest': {'lat': 36.2057908197085,
                                          'lng': 58.7895812697085}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/hospital-71.png',
  'icon_background_color': '#F88181',
  'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hospital-H_pinlet',
  'name': '22 Bahman Hospital',
  'opening_hours': {'open_now': True},
  'photos': [{'height': 3456,
              'html_attributions': ['<a '
                                    'href="https://maps.google.com/maps/contrib/112912241286164966057">Mehdi '
                                    'Ghodsi</a>'],
              'photo_reference': 'Aap_uEAMjuhEEAhPzie4BnZ1BB_uk8O8_ALo9WQIrELGMB6puxo4cu8w

{'html_attributions': [], 'next_page_token': 'Aap_uEBiYsb0ZqRwfNAnYJ5yHcqjm0vGUDPTvClxA3uVZNx54bxfYDtVjrX0QIrOz6l6BlM73CP38aKra6EeapvVAwfL8wW8lhhX2tTBh89H7_SNvxognyZEEhLG5xuYVW1VkOZWJknx6LA1OJU-hsedoCMtUH5UFLY89jgllCq3nVMCxs7CkPqrvmZDItmrIbX1WPZ7WC8eyKf1OGGhl4J3dXMja1OABjHoE4ckZ5_F0eIiP3OcmT4v_NFlyv6-Z8vjKtVb8DI-vJN6Fsq8vO9guToiWT3OcInQmKTvi85GhnqUNBW0jKzjwm8Wp1T6v4HnJ7oJShfcr32_lvlP-kCY6x-COZN2BowHvbQLnVAJb7ETc7CV0D8EeuGwH341QUWxSEMLKSgUqVnaDsJgmxATj7fa-FPM9H-VnllTjAwOwPN2tn1bbYXErQ', 'results': [{'geometry': {'location': {'lat': -8.753404199999999, 'lng': -67.39734779999999}, 'viewport': {'northeast': {'lat': -8.74237600998174, 'lng': -67.38134037622858}, 'southwest': {'lat': -8.764432063186145, 'lng': -67.41335522377142}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png', 'icon_background_color': '#7B9EB0', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_pinlet', 'name': 'Boca do Acre', 'photos': [{'heig

  'plus_code': {'compound_code': '6JX2+8M Centro, Boca do Acre - AM, Brazil',
                'global_code': '673J6JX2+8M'},
  'reference': 'ChIJT2NHVLeggJER7wYS9AU-uK0',
  'scope': 'GOOGLE',
  'types': ['point_of_interest', 'store', 'establishment'],
  'vicinity': 'Centro, Boca do Acre'},
 {'geometry': {'location': {'lat': -8.7725226, 'lng': -67.33219919999999},
               'viewport': {'northeast': {'lat': -8.742357900747017,
                                          'lng': -67.28693165431211},
                            'southwest': {'lat': -8.84418412153882,
                                          'lng': -67.37673492629062}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
  'icon_background_color': '#7B9EB0',
  'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_pinlet',
  'name': 'Platô do Piquiá',
  'photos': [{'height': 4160,
              'html_attributions': ['<a '
                          

In [95]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel Name
15,Saskylakh,71.9167,114.0833,76.60,40,0,4.34,RU,1627864110,Saskylakh
160,Abu Samrah,35.3029,37.1841,77.86,45,0,6.64,SY,1627863934,Suruj
245,Nova Olímpia,-14.7972,-57.2881,72.50,32,0,3.15,BR,1627864080,Nova Olímpia
247,Pontes e Lacerda,-15.2261,-59.3353,70.84,31,0,5.68,BR,1627864198,Centro
339,Damietta,31.4165,31.8133,79.65,67,0,6.08,EG,1627864237,Damietta
379,Gaoua,10.2992,-3.2508,71.67,88,0,4.43,BF,1627864253,Sidoumoukar
468,Pula,44.8683,13.8481,74.26,82,0,3.44,HR,1627864002,Pula
506,Mahon,39.8885,4.2658,70.36,78,0,4.61,ES,1627864304,Mahón
513,Neyshabur,36.2133,58.7958,71.65,16,0,2.53,IR,1627864307,Neyshabur
516,Boca do Acre,-8.7522,-67.3978,73.67,50,0,3.56,BR,1627864308,Boca do Acre


In [96]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
#print(hotel_info)

In [97]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add layer
fig.add_layer(markers)

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))